### Absolute Beginners Tutorial for MPI

Today we'll review an absolute beginner's use of MPI and DeepSpeed.  For this tutorial we'll use OpenMPI which is a message passing interface often used at super-computing centers for high performance computing.  It can be used for problems that require parallelism or those that require distributed computing (where processes require careful coordination).  OpenMPI on Domino Data Lab is easy to use.  

Here are a few tips on hardware choices:

1. Make sure to choose the correct hardware.  If working with small to medium size data (under a gigabyte) often a small or medium hardware tier is sufficient.
2. If using large datasets (larger than a GB) it is often useful to select a high memory hardware tier in either a CPU or GPU.
3. If using a complex calculation with very large data (larger than a few GB) then hardware 

For DeepSpeed use cases, the models and / or data is typically large, so a high-memory, GPU is most helpful.

### What is OpenMPI?

A message passing interface (MPI) is a computer program that passes communications between hardware on a cluster and helps to manage shared memory.

The Open MPI Project is an open-sourced MPI implementatation that is created and maintained by a group of academic, research and industry partners.  Its a popular choice for Higher Performance omputing and supercomputing centers. 

Some of the features of OpenMPI include:

- Conforms to MPI-3.1 standards
- Thread management and concurrency (your computer will not get 'fried')
- Support of all networks
- Supports most job schedulers
- High Performance on all platforms
- Open source license (BSD license)

### How MPI is used in High Performance Computing

High Performance Computing (HPC) is simply -- at high speeds -- performing compex calculations.  It run both parallel and distributed problems.  The best known example of HPC is a supercomputer.  Supercomputers are made up of many CPUs or GPUs and processors working together to perform both parallel and distributed tasks.  MPI aids in this by coorindating messaging among CPUs, GPUs and nodes.

### What the learner will be able to do at the end of this tutorial:

Upon completion of part one of this tutorial the learner will be able to:
    
- Describe OpenMPI and its use cases.
- Identify whether code is running with parallelism or distributed across a cluster.
- Perform a distributed calculation of pi using MPI workers.
- Explain the use of python files to run programs on a MPI-managed cluster.
- Determine what commands are necessary in mpi4py to distribute calculations across a cluster.

#### A 'Hello World' Example

In order to run a program or python files using MPI and python the code in the file needs to be set-up to run on the cluster.  In this exercise we will look at the number of processes running and the rank for each.  This ``hello world`` example is run from a sepparate python file.  One can run many python programs in the same manner.

Note: make sure to sync your MPI cluster before running new code.

```
from __future__ import print_function
from mpi4py import MPI
comm = MPI.COMM_WORLD
print("Hello! I'm rank %d from %d running in total..." % (comm.rank, comm.size))
comm.Barrier()

```

Notice in the code above we set up the communicator, which indicates the 'world' size or number of cpus or gpus in the cluster on which the program will run.  The ```comm.Barrier()``` function tells the program to wait until all processes and workers are synced.  Once the code is properly formed the program can be run at the command line using ``mpirun`` along with the appropriate options.  Try this for yourself changing the number of processes.  You can also add or change the 'hello world' code to run a different python function.  What happens when you change the number of processes (``-np``)?  Does it go faster, slower or the same?  How does the print-out change?

In [ ]:
!mpirun hostname

In [ ]:
!mpirun -np 1  python hello_world.py

Once can also use the command ``mpiexec`` as an execution command for a python file.  There is very little difference between ``mpirun`` and ``mpiexec``.  Feeding the hostfile location to the mpi run will make explicit to the mpi run, the location of each worker.  This facilitates the distribution of calculations or runs across workers.

In [ ]:
!mpiexec --hostfile /domino/mpi/hosts -np 1 python hello_world.py 

#### Calculating Pi on Worker Processes

Below is the code we use to run a generic python file which calculates pi.  You may notice that there is little difference in the time to run or the output of the cpi2.py file.  This is in part because we have not broadcasted the data to the workers, thus we are not running the file in a distributed manner.  Rather the file is just running once on each worker process indicated.  Keep reading, we will look at how to distribute a calculation over the mpi workers with the python library mpi4py.  Try experimenting with the number of processes used (np) in the mpirun.  You'll see that each change in process number simply runs it on that number of workers, but in a parallel manner, not distributed.

The code in the cpi2.py file is as such:

```
import numpy as np
import math

N=10**8

h = 1.0 / N; s = 0.0
for i in range(N):
    x = h * (i + 0.5)
    s += 4.0 / (1.0 + x**2)
    estimated_pi = s * h

print(estimated_pi)
```

In [ ]:
%%time

!python cpi2.py

In [ ]:
%%time

!mpirun -np 1 --hostfile /domino/mpi/hosts --bind-to none --map-by slot python cpi2.py

Notice that the code when run using ``mpirun`` was not significantly faster than running as a vanilla python file.  Why do you think this is?  Hint: there's a difference between simple parallelism and distributed functions. Try changing the number of processes again.  You'll notice the caluclation of pi prints out the same number of times as the number of processes.  Why do you think that is?

The MPI program also has an easy command to call a list of options for ``mpirun`` and other mpi commands.  See the example below.

In [ ]:
### how to reach the mpi help file

!mpirun --help

#### Calculating Pi with and without Distributed Compute

Below let's look at an example of how to calculate pi without mpi (naïve method) and with mpi (distributed method).  The formula we'll use to calculate pi is simple. The number pi is a ratio obtained from defining the area with a circle. If the diameter and the circumference of a circle are known, the value of pi will be as π = circumference of a circle/diameter of a circle.  In our calculation we take a number of samples ``N`` to estimate pi more accurately and to test whether distributing these calculation over workers is more efficient.  This is basically the Leibniz formula which you can find out more about it [here](https://en.wikipedia.org/wiki/Leibniz_formula_for_%CF%80).

A more complex example of calculating pi uses a monte carlo calculation and the code is available in the supplemental materials.  That example runs on ``mpi`` using the python library, ``mpi4py``.

In [ ]:
%%time

import numpy as np
import math

N=10**8

h = 1.0 / N; s = 0.0
for i in range(N):
    x = h * (i + 0.5)
    s += 4.0 / (1.0 + x**2)
    estimated_pi = s * h

print(estimated_pi)

We can also run this code in a python file using ``mpirun``.

In [ ]:
%%time

import numpy as np
import math

!mpirun python /mnt/cpi2.py

Now let's compare that calculation to calculating pi using OpenMPI.  For this function we will use a python wrapper / library around MPI called ``mpi4py``.  We will keep our calculation code in a sepparate file so we can run the function over MPI.

#### Client Side Code

The client-side code will output our calcuation of pi from the file cpi.py.  We will take a look at the client side code that is in the cpi.py file after we run the code below.

In [ ]:
## prepare for distributed calculation using MPI

maxprocs = input('How many GPU workers do you have?')

In [ ]:
%%time

from mpi4py import MPI
import numpy
import sys
import mpi4py



comm = MPI.COMM_SELF.Spawn(sys.executable,
                           args=['cpi.py'],
                           maxprocs=int(maxprocs))

N = numpy.array(10**8, 'i')
comm.Bcast([N, MPI.INT], root=MPI.ROOT)
PI = numpy.array(0.0, 'd')
comm.Reduce(None, [PI, MPI.DOUBLE],
            op=MPI.SUM, root=MPI.ROOT)
print(PI)

comm.Disconnect()

The code above uses ``Spawn`` to initiate the communications across a cluster, the ``Bcast`` to broadcast the calculation and numpy array across workers and use the function ``Reduce`` to reduce the calculations across workers to a final answer that will be printed.

#### Server Side Code (contained in cpi.py)

Notice the code below contains our generic calculation of pi using the Lebniz model. Consider using the paradigm with mpi4py in which the calculations will be truly distributed via broadcating to the workers and reducing the worker's calculations to a final answer.  Optionally you can put both server-side and client side code into one file.  Use the instructions earlier to run the file on the cluster.

The server-side code for the MPI example above looks like this:

```
#!/usr/bin/env python
from mpi4py import MPI
import numpy

comm = MPI.Comm.Get_parent()
size = comm.Get_size()
rank = comm.Get_rank()
name = MPI.Get_processor_name()

N = numpy.array(0, dtype='i')
comm.Bcast([N, MPI.INT], root=0)
h = 1.0 / N; s = 0.0
for i in range(rank, N, size):
    x = h * (i + 0.5)
    s += 4.0 / (1.0 + x**2)
PI = numpy.array(s * h, dtype='d')
comm.Reduce([PI, MPI.DOUBLE], None,
            op=MPI.SUM, root=0)

comm.Disconnect()

```
We see the instantiation of the following variables:

- comm - the parent communication worker (usually worker 0)
- size - the size of the cluster
- rank - the rank of each GPU worker in the cluster (example if there are five the ranks are 0, 1, 2, 3, 4)
- name - the processor name

These variables are required so that messages can be passed between workers / shared memory and a reduction operation can present the end results of our calculation of pi.

#### What we learned in this tutorial:

- OpenMPI stands for the open-source version of the message passing interface software.

- MPI can be used to run generic python files using ``mpirun`` or ``mpiexec``.

- Care must be taken to run code in a distributed manner on a cluster rather than running solely in parallel.

- Code changes are required with the use of the ``mpi4py`` library in order to distribute data or calculations.

- Choosing the correct hardware will make sure calculations run smoothly.


#### To learn more see these references and tutorials:
    
[Basics of MPI](https://carleton.ca/rcs/rcdc/introduction-to-mpi)

[OpenMPI Documentation](https://www.open-mpi.org/faq/?category=running#simple-launch)

[Parallel Programming with MPI and Python](https://rabernat.github.io/research_computing/parallel-programming-with-mpi-for-python.html)